# Source

[Predict your Wine Quality using Deep Learning with PyTorch](https://medium.com/@srijaneogi31/predict-your-wine-quality-using-deep-learning-with-pytorch-424d736f0880)

# Libraires

In [83]:
# !pip install plotly -U --quiet

In [84]:
import  matplotlib.pyplot as plt
import  numpy as np
import  pandas as pd
import  torch
import  torchvision
import  matplotlib
import  torch.nn as nn
import  pandas as pd
import  matplotlib.pyplot as plt
import  seaborn as sns
import  torch.nn.functional as F


from    torchvision.datasets.utils import download_url
from    torch.utils.data import DataLoader, TensorDataset, random_split
from    google.colab import drive

In [85]:
seed = np.random.seed(31)

In [86]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dataset

In [87]:
df_wine = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/winequality-red.csv',
                      sep=',',
                      encoding='utf-8')

df_wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [88]:
df_wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [89]:
features = df_wine.drop('quality', axis=1).columns.to_list()

features

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [90]:
target = [df_wine.columns.to_list()[-1]]

target

['quality']

In [91]:
# df_wine[target].replace([3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5],
#                            inplace=True)

In [92]:
df_wine.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5


# Tensors

In [93]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)

    # Convert non-numeric categorical columns to numbers
    # for col in categorical_cols:
    #     dataframe1[col] = dataframe1[col].astype('category').cat.codes

    # Extract input & outupts as numpy arrays
    features_array = dataframe1[features].to_numpy()

    target_array = dataframe1[target].to_numpy()

    return features_array, target_array

In [94]:
features_array, target_array = dataframe_to_arrays(df_wine)

In [95]:
features_tensor = torch.from_numpy(features_array).type(torch.float)

target_tensor = torch.LongTensor(target_array)

features_tensor, target_tensor

(tensor([[ 7.4000,  0.7000,  0.0000,  ...,  3.5100,  0.5600,  9.4000],
         [ 7.8000,  0.8800,  0.0000,  ...,  3.2000,  0.6800,  9.8000],
         [ 7.8000,  0.7600,  0.0400,  ...,  3.2600,  0.6500,  9.8000],
         ...,
         [ 6.3000,  0.5100,  0.1300,  ...,  3.4200,  0.7500, 11.0000],
         [ 5.9000,  0.6450,  0.1200,  ...,  3.5700,  0.7100, 10.2000],
         [ 6.0000,  0.3100,  0.4700,  ...,  3.3900,  0.6600, 11.0000]]),
 tensor([[5],
         [5],
         [5],
         ...,
         [6],
         [5],
         [6]]))

In [96]:
dataset_tensor = TensorDataset(features_tensor, target_tensor)

In [97]:
print('Shape of input tensor and target tensor::  ', features_tensor.shape, target_tensor.shape)
print('datatype of input tensor and target tensor::  ', features_tensor.dtype, target_tensor.dtype)

Shape of input tensor and target tensor::   torch.Size([1599, 11]) torch.Size([1599, 1])
datatype of input tensor and target tensor::   torch.float32 torch.int64


# Train test split

In [98]:
train_size = int(len(dataset_tensor) * 0.8)

val_size = int(len(dataset_tensor) - train_size)

train_size, val_size

(1279, 320)

In [99]:
train, val = random_split(dataset_tensor, [train_size, val_size])

batch_size=50

train_loader = DataLoader(train, batch_size, shuffle=True)

val_loader = DataLoader(val, batch_size)

# Linear Regression Model

In [100]:
input_size = len(features)

output_size = len(target)

input_size, output_size

(11, 1)

In [101]:
class WineQuality(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)  #???                  # fill this (hint: use input_size & output_size defined above)

    def forward(self, xb):
        out = self.linear(xb)  #???

        return out

    def training_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calcuate loss
        loss = F.l1_loss(out,targets)  #???

        return loss

    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out,targets)  #???

        return {'val_loss': loss.detach()}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses

        return {'val_loss': epoch_loss.item()}

    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 100th epoch
        if (epoch+1) % 100 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [102]:
model = WineQuality()

In [103]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]

    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):

        # Training Phase
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)


    return history

## Train model

In [104]:
epochs = 1500

learning_rate = 1e-4

history = fit(epochs, learning_rate, model, train_loader, val_loader)

Epoch [100], val_loss: 0.5960
Epoch [200], val_loss: 0.5738
Epoch [300], val_loss: 0.5474
Epoch [400], val_loss: 0.5365
Epoch [500], val_loss: 0.5277
Epoch [600], val_loss: 0.5208
Epoch [700], val_loss: 0.5183
Epoch [800], val_loss: 0.5269
Epoch [900], val_loss: 0.5150
Epoch [1000], val_loss: 0.5149
Epoch [1100], val_loss: 0.5127
Epoch [1200], val_loss: 0.5111
Epoch [1300], val_loss: 0.5149
Epoch [1400], val_loss: 0.5249
Epoch [1500], val_loss: 0.5241


## Predictions

In [113]:
def predict_single(input, target, model):
    inputs = input
    predictions = model(inputs)  #???
    prediction = predictions[0].detach()

    print('Input:', input)
    print('Target:', target)
    print('Prediction:', prediction)

In [114]:
input, target = val[62]

predict_single(input, target, model)

Input: tensor([ 8.7000,  0.4800,  0.3000,  2.8000,  0.0660, 10.0000, 28.0000,  0.9964,
         3.3300,  0.6700, 11.2000])
Target: tensor([7])
Prediction: tensor(6.0461)
